# Notebook 5: Fate Matrices

In this notebook we show how to compute and visualize 'fates' of cells. 
Consider a pair of time points $t_i < t_j$ and a collection of cell sets $C_1,\ldots,C_k$ spanning all cells at the later time point, $t_j$. (That is, each cell $y$ from time $t_j$ is in some $C_\ell$). 
We show how to compute the probability that a cell $x$ from time $t_i$ will transition to a cell set $C_\ell$ at time $t_j$. 
We call these the fate probabilities for cell $x$. The **fate matrix** $F_{t_i,t_j}$ is a matrix with a row containing the fate probabilities for each cell $x$ from time $t_i$.

In [1]:
# if you haven't yet installed wot, run the line below
#!pip install -q wot
import ipywidgets as widgets
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import wot

In [2]:
# Load transport map model and cell sets
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps/serum')
cell_sets = wot.io.read_sets('data/cell_sets.gmt', as_dict=True)

# create indicator vectors for each cell set
target_fates = tmap_model.population_from_cell_sets(cell_sets, at_time=18)

## Compute fates
In the cell below, we use the function `fates` to compute fate matrices for all earlier time points. These are concatentated in the `fate_ds` dataset. We can then easily subset this dataset to access the fate matrix $F_{t_i,t_j}$ for any individual day $t_i$.

In [3]:
fate_ds = tmap_model.fates(target_fates)

## Visualize fates

In [4]:
fate_dropdown1 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 1:'
)
fate_dropdown2 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 2:',
    value=fate_ds.var.index[1]
)

def update_fate_vis(name1, name2):
    figure = plt.figure(figsize=(10, 10))
    fate1 = fate_ds[:, name1].X
    fate2 = fate_ds[:, name2].X
    p = np.log(1e-9 + np.divide(fate1, fate2, out=np.zeros_like(fate1), where=fate2 != 0))
    plt.scatter(fate_ds.obs['day'], p, s=4, marker=',')
    plt.xlabel('Day')
    plt.ylabel('Log Odds')
    plt.title('{} vs. {}'.format(name1, name2))

widgets.interact(update_fate_vis, name1=fate_dropdown1, name2=fate_dropdown2)

interactive(children=(Dropdown(description='Fate 1:', options=('IPS', 'Stromal', 'Neural', 'Epithelial', 'Trop…

<function __main__.update_fate_vis(name1, name2)>